In [1]:
import pandas as pd
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')


data = pd.read_csv('data/train.csv')

# Display the first few rows of the dataset
data.head()

In [2]:
import datetime
import numpy as np

# Convert day of the year to month
data['month'] = data['day'].apply(lambda x: datetime.datetime.strptime(str(x), '%j').month)
data['month'] = np.sin(2 * np.pi * data['month'] / 12)

# Display the updated dataframe
data.tail()

In [3]:
# Drop columns 'id' and 'day', and set 'rainfall' as the label column
X = data.drop(columns=['id', 'day', 'rainfall'])
y = data['rainfall']


In [4]:
# from sklearn.preprocessing import StandardScaler

# # Initialize the StandardScaler
# scaler = StandardScaler()

# # Fit and transform X
# X = scaler.fit_transform(X)

In [5]:
from sklearn.model_selection import train_test_split

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

In [6]:
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from imblearn.combine import SMOTEENN
# from sklearn.metrics import roc_auc_score

# # Handle class imbalance
# # Combination of over- and under-sampling using SMOTEENN


# smoteenn = SMOTEENN(random_state=42)
# X_train_resampled, y_train_resampled = smoteenn.fit_resample(X_train, y_train)

# # Define models
# models = {
#     "Random Forest": RandomForestClassifier(random_state=42),
#     "Gradient Boosting": GradientBoostingClassifier(random_state=42),
# }

# # Train and validate each model
# for name, model in models.items():
#     model.fit(X_train_resampled, y_train_resampled)
#     y_test_pred_proba = model.predict_proba(X_test)[:, 1]
#     auc_score = roc_auc_score(y_test, y_test_pred_proba)
#     print(f"Model: {name}")
#     print(f"AUC-ROC Score: {auc_score:.4f}")
#     print("-" * 50)




In [7]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import VotingClassifier
# from sklearn.metrics import classification_report


# # Perform hyperparameter tuning for the top 3 models
# tuned_models = {}
# param_grids = {
#     "Random Forest": {
#         "n_estimators": [100, 200],
#         "max_depth": [None, 10, 20],
#         "min_samples_split": [2, 5],
#     },
#     "Gradient Boosting": {
#         "n_estimators": [100, 200],
#         "learning_rate": [0.1, 0.01],
#         "max_depth": [3, 5],
#     },
# }

# for name, model in list(models.items()):
#     print(f"Tuning {name}...")
#     grid_search = GridSearchCV(model, param_grids[name], scoring="f1_weighted", cv=3, n_jobs=-1)
#     grid_search.fit(X_train_resampled, y_train_resampled)
#     tuned_models[name] = grid_search.best_estimator_
#     print(f"Best parameters for {name}: {grid_search.best_params_}")





In [9]:
from ndf import train_neural_forest_binary
import torch
# Convert to PyTorch tensors
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)  # Use long for classification labels

ndf_model, _, metrics = train_neural_forest_binary(X_tensor, y_tensor, input_dim=11)

# Display the metrics
print(metrics)


In [61]:
# Create an ensemble model using the tuned models

ensemble_model = VotingClassifier(
    estimators=list(tuned_models.items()),
    voting="soft"
)
ensemble_model.fit(X_train_resampled, y_train_resampled)

# Step 4: Test the ensemble model on the test set
y_test_pred = ensemble_model.predict(X_test)
# Calculate and display the AUC-ROC score for the ensemble model
y_test_pred_proba = ensemble_model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_test_pred_proba)
print(f"\nEnsemble Model AUC-ROC Score on Test Set: {auc_score:.4f}")

In [1]:
# Read the test data
test_data = pd.read_csv('data/test.csv')
test_data['winddirection'].fillna(test_data['winddirection'].mean(), inplace=True)

# Extract the 'id' column for the submission file
test_ids = test_data['id']

# Drop unnecessary columns to match the training features
# Ensure the columns 'id' and 'day' exist before dropping them
if 'id' in test_data.columns and 'day' in test_data.columns:
    X_test_final = test_data.drop(columns=['id', 'day'])
else:
    raise ValueError("The columns 'id' and 'day' are missing from the test dataset.")

# Predict the probability of rainfall using the ensemble model
rainfall_probabilities = ensemble_model.predict_proba(X_test_final)[:, 1]

# Create the submission dataframe
submission = pd.DataFrame({
    'id': test_ids,
    'rainfall': rainfall_probabilities
})

# Write the submission to a CSV file
submission.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' has been created.")

NameError: name 'pd' is not defined